In [ ]:
using JuMP, Gurobi, DataFrames, CSV, Plots, StatsPlots, LinearAlgebra, Random, StatsBase, Statistics, Clustering

## <u>Import Datasets<u>___________________________________________

In [ ]:
# Read in locations, emissions, dist, waste_per_day, cost and take columns 2:end for matrices
locations_df = CSV.read("locations.csv", DataFrame)

emissions_df = CSV.read("emissions.csv", DataFrame)
emissions_df = emissions_df[:,2:end]

# rename columns to be 1 to 15
rename!(emissions_df, Symbol.(1:15))

dist_df = CSV.read("dist.csv", DataFrame)
dist_df = dist_df[:,2:end]

# rename columns to be 1 to 15
rename!(dist_df, Symbol.(1:15))

waste_df = CSV.read("waste_per_day.csv", DataFrame)

cost_df = CSV.read("cost.csv", DataFrame)
cost_df = cost_df[:,2:end]

# rename columns to be 1 to 15
rename!(cost_df, Symbol.(1:15))

Row,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,61.8162,60.3871,62.2014,61.4747,60.5924,62.4447,61.7531,63.6153,64.3073,63.1925,65.7736,65.1711,63.9061,0.0
2,61.8162,0.0,1.90745,1.48735,1.85797,3.21045,3.06771,3.59665,4.11128,5.57575,5.98148,6.95952,7.88149,7.76994,61.8162
3,60.3871,1.90745,0.0,1.82078,1.21665,1.69546,2.66789,2.66818,4.00629,5.35129,5.28343,6.92752,7.48254,7.05438,60.3871
4,62.2014,1.48735,1.82078,0.0,0.829276,2.23779,1.58707,2.21758,2.65842,4.1185,4.49831,5.56094,6.41015,6.28392,62.2014
5,61.4747,1.85797,1.21665,0.829276,0.0,1.45097,1.51132,1.78303,2.81896,4.20863,4.31628,5.75517,6.41178,6.11282,61.4747
6,60.5924,3.21045,1.69546,2.23779,1.45097,0.0,1.85232,1.30392,3.09185,4.17825,3.79459,5.80044,6.09044,5.50388,60.5924
7,62.4447,3.06771,2.66789,1.58707,1.51132,1.85232,0.0,0.924401,1.34865,2.6981,2.9138,4.25992,4.91351,4.70454,62.4447
8,61.7531,3.59665,2.66818,2.21758,1.78303,1.30392,0.924401,0.0,1.86274,2.87449,2.6153,4.49659,4.84832,4.39079,61.7531
9,63.6153,4.11128,4.00629,2.65842,2.81896,3.09185,1.34865,1.86274,0.0,1.46448,2.20972,2.93953,3.77725,3.84738,63.6153


By virtue of how indexing works in Julia, the following indices now denote Neighbourhoods and Landfill respectively
- **Landfill**: [1] and [15]
- **Neighbourhoods**: [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14]

Hence, the following **Sets** are applicable to the problem
- **Neighbourhoods**: *i* = 2, ... , 14
- **Neighbourhoods**: *j* = 2, ... , 14
- **Landfill**: *i* = 1 and 15, *j* = 1 and 15
- **Vehicles**: *k* = 1, 2, 3

In [ ]:
# append waste_df 3rd column to locations_df
locations_df[!, :waste] = waste_df[:,3]

# create new column in locations_df to denote number of trucks needed to service each neighborhood
locations_df[!, :n_trucks] = zeros(size(locations_df,1))

# compute number of trucks needed to service each neighborhood: n_trucks = ceil(waste produced per day/capacity)
capacity_in_tons = 14

for i = 1:size(locations_df,1)
    locations_df[i,:n_trucks] = locations_df[i,6]/(capacity_in_tons*907.185)
end

# sum the n_trucks column of locations_df
n_vehicles_needed = sum(locations_df[!, :n_trucks])
locations_df

Row,Neighborhood,Area,2020 Households,Centerpoint X,Centerpoint Y,waste,n_trucks
,String31,Int64,Int64,Float64,Float64,Float64,Float64
1,Landfill,0,0,-71.0864,41.9235,0.0,0.0
2,East Cambridge,1,6953,-71.079,42.3683,47975.7,3.77744
3,MIT,2,806,-71.0914,42.358,5561.4,0.437885
4,Wellington-Harrington,3,2931,-71.093,42.371,20223.9,1.59236
5,The Port,4,2890,-71.0968,42.3658,19941.0,1.57008
6,Cambridgeport,5,5279,-71.1078,42.3592,36425.1,2.86798
7,Mid-Cambridge,6,6460,-71.1084,42.3725,44574.0,3.5096
8,Riverside,7,4047,-71.1141,42.3674,27924.3,2.19866
9,Baldwin,8,1799,-71.1154,42.3807,12413.1,0.977364


In [ ]:
# how many vehicles are needed to service all neighborhoods?
n_vehicles_needed = sum(locations_df[!, :n_trucks])
println("Capacity of an individual truck is ", capacity_in_tons*907.185, " kg")
println("With a capacity of ", capacity_in_tons*907.185, " kg per truck, we need ", n_vehicles_needed, " individual trucks to service all neighborhoods.")

# find maximum waste produced per day
max_waste = maximum(locations_df[!, :waste])
n_individual_per_aggregate = ceil(max_waste/(capacity_in_tons*907.185))

println("With a capacity of ", capacity_in_tons*907.185, " kg per truck, we need ", n_individual_per_aggregate, " individual trucks to form one aggregated truck")
# use "aggregated" form of trucks - each "aggregated truck" has a capacity of 14*4 = 56 tons
println("Capacity of an aggregated truck = ", capacity_in_tons*907.185*n_individual_per_aggregate, " kg")
println("We need ", ceil(n_vehicles_needed/n_individual_per_aggregate), " aggregated trucks to service all neighborhoods.")

Capacity of an individual truck is 12700.59 kg
With a capacity of 12700.59 kg per truck, we need 26.927221491285053 individual trucks to service all neighborhoods.
With a capacity of 12700.59 kg per truck, we need 4.0 individual trucks to form one aggregated truck
Capacity of an aggregated truck = 50802.36 kg
We need 7.0 aggregated trucks to service all neighborhoods.


In [ ]:
# convert cost matrix to 3D array
n_vehicles = ceil(n_vehicles_needed/n_individual_per_aggregate) + 1

# convert n_vehicles to Int64
n_vehicles = Int64(n_vehicles)
cost = zeros(15,15,n_vehicles)

for i = 1:size(cost,1)
    for j = 1:size(cost,2)
        for k = 1:n_vehicles
            cost[i,j,k] = n_individual_per_aggregate*cost_df[i,j]
        end
    end
end

# extract waste per day
waste = zeros(15)

for i = 1:length(waste)
    waste[i] = waste_df[i,end]
end

In [ ]:
# convert emissions matrix to 3D array
emissions = zeros(15,15,n_vehicles)

for i = 1:size(emissions,1)
    for j = 1:size(emissions,2)
        for k = 1:n_vehicles
            emissions[i,j,k] = n_individual_per_aggregate*emissions_df[i,j]
        end
    end
end

## <u>Emissions and Cost Together (Waste Management 5)

In [ ]:
function waste_management5(λ, c, e, w, n_vehicles, n_individual_per_aggregate)

    # compute capacity
    cap = ceil(n_individual_per_aggregate*14*907.185)

    # create model
    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "TimeLimit", 300)

    # create variables
    @variable(model, x[i=1:15, j=1:15, k=1:n_vehicles], Bin)
    @variable(model, u[i=1:15, k=1:n_vehicles])

    # objective function
    @objective(model, Min, sum(λ*c[i,j,k]*x[i,j,k] + (1-λ)*e[i,j,k]*x[i,j,k] for i=1:15, j=1:15, k=1:n_vehicles))

    # constraints
    ## 1) No travel from a node to itself
    @constraint(model, [i=1:15, k=1:n_vehicles], x[i,i,k] == 0)

    ## 2) No travel from landfill node directly back to landfill node
    @constraint(model, [k=1:n_vehicles], x[1,15,k] == 0)

    ## 3) No outflow from node 15: Once arrived at node 15 (landfill), cannot leave
    @constraint(model, [j=1:15, k=1:n_vehicles], x[15,j,k] == 0)

    ## 4) No inflow into node 1: Can only depart from node 1 (landfill), not arrive
    @constraint(model, [i=1:15, k=1:n_vehicles], x[i,1,k] == 0)

    ## 5) Every vehicle must depart from node 1 (landfill)
    @constraint(model, [k=1:n_vehicles], sum(x[1,j,k] for j=2:14) == 1)

    ## 6) Every neighbourhood j is visited exactly once across all vehicles (for all neighbourhood nodes 2 to 14)
    @constraint(model, [j=2:14], sum(sum(x[i,j,k] for i=1:14) for k=1:n_vehicles) == 1)

    ## 7) Every vehicle must return to node 15 (landfill)
    @constraint(model, [k=1:n_vehicles], sum(x[i,15,k] for i=2:14) == 1)

    ## 8) Vehicle capacity cannot be exceeded throughout journey
    @constraint(model, [k=1:n_vehicles], sum(sum(w[j]*x[i,j,k] for j=2:15) for i=1:14) <= cap)

    ## 9) MTZ subtour elimination
    @constraint(model, [i=1:14, j=2:15, k=1:n_vehicles], u[j,k] >= u[i,k] + w[j] - cap*(1-x[i,j,k]))
    @constraint(model, [i=2:15, k=1:n_vehicles], u[i,k] >= w[i])
    @constraint(model, [i=2:15, k=1:n_vehicles], u[i,k] <= cap)
    @constraint(model, [k=1:n_vehicles], u[1,k] == 0)

    ## 10) Every vehicle k must depart from the node i that it entered
    @constraint(model, [i=2:14, k=1:n_vehicles], sum(x[j,i,k] for j=1:14) == sum(x[i,j,k] for j=2:15))

    # solve model
    optimize!(model)

    # return decision variables
    return value.(x), value.(u), objective_value(model)

end

waste_management5 (generic function with 1 method)

In [ ]:
# solve model (with varying lamdas)
λ = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

# initialise an empty matrix to store results for total_cost, total_emissions, and objective_value
results = zeros(length(λ),4)

for lambdaindex = 1:length(λ)
    x_opt_5, u_opt_5, opt_sol = waste_management5(λ[lambdaindex], cost, emissions, waste, n_vehicles, n_individual_per_aggregate);

    # calculate cost using cost*x_opt_5
    total_cost = sum(sum(sum(cost[i,j,k]*x_opt_5[i,j,k] for i=1:15) for j=1:15) for k=1:n_vehicles)

    # calculate emissions using emissions*x_opt_5
    total_emissions = sum(sum(sum(emissions[i,j,k]*x_opt_5[i,j,k] for i=1:15) for j=1:15) for k=1:n_vehicles)

    # save each lambda and its results
    results[lambdaindex,1] = λ[lambdaindex]
    results[lambdaindex,2] = total_cost
    results[lambdaindex,3] = total_emissions
    results[lambdaindex,4] = opt_sol

end

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter TimeLimit to value 300
Set parameter TimeLimit to value 300
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2301 rows, 1920 columns and 9464 nonzeros
Model fingerprint: 0x216ef5ec
Variable types: 120 continuous, 1800 integer (1800 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+04]
  Objective range  [7e+00, 5e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+04]
Presolve removed 1608 rows and 1064 columns
Presolve time: 0.02s
Presolved: 693 rows, 856 columns, 6966 nonzeros
Variable types: 88 continuous, 768 integer (768 binary)

Root relaxation: objective 8.028474e+03, 228 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestB

In [ ]:
# show results_df
results

11×4 Matrix{Float64}:
 0.0  4049.52  8099.04  8099.04
 0.1  4049.52  8099.04  7694.09
 0.2  4049.52  8099.04  7289.13
 0.3  4049.52  8099.04  6884.18
 0.4  4049.52  8099.04  6479.23
 0.5  4049.52  8099.04  6074.28
 0.6  4049.52  8099.04  5669.33
 0.7  4049.52  8099.04  5264.37
 0.8  4049.52  8099.04  4859.42
 0.9  4049.52  8099.04  4454.47
 1.0  4049.52  8099.04  4049.52

In [ ]:
x_opt_5, u_opt_5, opt_sol = waste_management5(0.1, cost, emissions, waste, n_vehicles, n_individual_per_aggregate);


Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter TimeLimit to value 300
Set parameter TimeLimit to value 300
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2301 rows, 1920 columns and 9464 nonzeros
Model fingerprint: 0xd3f87c82
Variable types: 120 continuous, 1800 integer (1800 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+04]
  Objective range  [6e+00, 5e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+04]
Presolve removed 1608 rows and 1064 columns
Presolve time: 0.02s
Presolved: 693 rows, 856 columns, 6966 nonzeros
Variable types: 88 continuous, 768 integer (768 binary)

Root relaxation: objective 7.627051e+03, 239 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestB

In [ ]:
sum(cost[i,j,k]*x_opt_5[i,j,k] for i=1:15, j=1:15, k=1:n_vehicles)

4049.518592912338

## <u>Emissions Minimisation (Waste Management 4)

In [ ]:
function waste_management5(e, w, n_vehicles, n_individual_per_aggregate)

    # compute capacity
    cap = ceil(n_individual_per_aggregate*14*907.185)

    # create model
    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "TimeLimit", 300)

    # create variables
    @variable(model, x[i=1:15, j=1:15, k=1:n_vehicles], Bin)
    @variable(model, u[i=1:15, k=1:n_vehicles])

    # objective function
    @objective(model, Min, sum(e[i,j,k]*x[i,j,k] for i=1:15, j=1:15, k=1:n_vehicles))

    # constraints
    ## 1) No travel from a node to itself
    @constraint(model, [i=1:15, k=1:n_vehicles], x[i,i,k] == 0)

    ## 2) No travel from landfill node directly back to landfill node
    @constraint(model, [k=1:n_vehicles], x[1,15,k] == 0)

    ## 3) No outflow from node 15: Once arrived at node 15 (landfill), cannot leave
    @constraint(model, [j=1:15, k=1:n_vehicles], x[15,j,k] == 0)

    ## 4) No inflow into node 1: Can only depart from node 1 (landfill), not arrive
    @constraint(model, [i=1:15, k=1:n_vehicles], x[i,1,k] == 0)

    ## 5) Every vehicle must depart from node 1 (landfill)
    @constraint(model, [k=1:n_vehicles], sum(x[1,j,k] for j=2:14) == 1)

    ## 6) Every neighbourhood j is visited exactly once across all vehicles (for all neighbourhood nodes 2 to 14)
    @constraint(model, [j=2:14], sum(sum(x[i,j,k] for i=1:14) for k=1:n_vehicles) == 1)

    ## 7) Every vehicle must return to node 15 (landfill)
    @constraint(model, [k=1:n_vehicles], sum(x[i,15,k] for i=2:14) == 1)

    ## 8) Vehicle capacity cannot be exceeded throughout journey
    @constraint(model, [k=1:n_vehicles], sum(sum(w[j]*x[i,j,k] for j=2:15) for i=1:14) <= cap)

    ## 9) MTZ subtour elimination
    @constraint(model, [i=1:14, j=2:15, k=1:n_vehicles], u[j,k] >= u[i,k] + w[j] - cap*(1-x[i,j,k]))
    @constraint(model, [i=2:15, k=1:n_vehicles], u[i,k] >= w[i])
    @constraint(model, [i=2:15, k=1:n_vehicles], u[i,k] <= cap)
    @constraint(model, [k=1:n_vehicles], u[1,k] == 0)

    ## 10) Every vehicle k must depart from the node i that it entered
    @constraint(model, [i=2:14, k=1:n_vehicles], sum(x[j,i,k] for j=1:14) == sum(x[i,j,k] for j=2:15))

    # solve model
    optimize!(model)

    # return decision variables
    return value.(x), value.(u)

end

waste_management4 (generic function with 1 method)

In [ ]:
x_emis_opt, u_emis_opt = waste_management4(emissions, waste, n_vehicles, n_individual_per_aggregate);

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter TimeLimit to value 300
Set parameter TimeLimit to value 300
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2309 rows, 1920 columns and 10920 nonzeros
Model fingerprint: 0x76add785
Variable types: 120 continuous, 1800 integer (1800 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+04]
  Objective range  [7e+00, 5e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+04]
Presolve removed 1608 rows and 1064 columns
Presolve time: 0.03s
Presolved: 701 rows, 856 columns, 7390 nonzeros
Variable types: 88 continuous, 768 integer (768 binary)

Root relaxation: objective 8.028474e+03, 297 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    Best

In [ ]:
# find where x_opt[:,:,k] is 1
x_emis_opt_1 = findall(x_emis_opt[:,:,8] .== 1)

4-element Vector{CartesianIndex{2}}:
 CartesianIndex(1, 3)
 CartesianIndex(3, 4)
 CartesianIndex(4, 5)
 CartesianIndex(5, 15)

In [ ]:
# print objective value of optimal solution
println("Optimal objective value = ", objective_value(model))

LoadError: UndefVarError: model not defined

## <u>Formulate Optimisation Problem #3<u>______________________________________

In [ ]:
function waste_management4(c, w, n_vehicles, n_individual_per_aggregate)

    # compute capacity
    cap = ceil(n_individual_per_aggregate*14*907.185)

    # create model
    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "TimeLimit", 300)

    # create variables
    @variable(model, x[i=1:15, j=1:15, k=1:n_vehicles], Bin)
    @variable(model, u[i=1:15, k=1:n_vehicles])

    # objective function
    @objective(model, Min, sum(c[i,j,k]*x[i,j,k] for i=1:15, j=1:15, k=1:n_vehicles))

    # constraints
    ## 1) No travel from a node to itself
    @constraint(model, [i=1:15, k=1:n_vehicles], x[i,i,k] == 0)

    ## 2) No travel from landfill node directly back to landfill node
    @constraint(model, [k=1:n_vehicles], x[1,15,k] == 0)

    ## 3) No outflow from node 15: Once arrived at node 15 (landfill), cannot leave
    @constraint(model, [j=1:15, k=1:n_vehicles], x[15,j,k] == 0)

    ## 4) No inflow into node 1: Can only depart from node 1 (landfill), not arrive
    @constraint(model, [i=1:15, k=1:n_vehicles], x[i,1,k] == 0)

    ## 5) Every vehicle must depart from node 1 (landfill)
    @constraint(model, [k=1:n_vehicles], sum(x[1,j,k] for j=2:14) == 1)

    ## 6) Every neighbourhood j is visited exactly once across all vehicles (for all neighbourhood nodes 2 to 14)
    @constraint(model, [j=2:14], sum(sum(x[i,j,k] for i=1:14) for k=1:n_vehicles) == 1)

    ## 7) Every vehicle must return to node 15 (landfill)
    @constraint(model, [k=1:n_vehicles], sum(x[i,15,k] for i=2:14) == 1)

    ## 8) Vehicle capacity cannot be exceeded throughout journey
    @constraint(model, [k=1:n_vehicles], sum(sum(w[j]*x[i,j,k] for j=2:15) for i=1:14) <= cap)

    ## 9) MTZ subtour elimination
    @constraint(model, [i=1:14, j=2:15, k=1:n_vehicles], u[j,k] >= u[i,k] + w[j] - cap*(1-x[i,j,k]))
    @constraint(model, [i=2:15, k=1:n_vehicles], u[i,k] >= w[i])
    @constraint(model, [i=2:15, k=1:n_vehicles], u[i,k] <= cap)
    @constraint(model, [k=1:n_vehicles], u[1,k] == 0)

    ## 10) Every vehicle k must depart from the node i that it entered
    @constraint(model, [i=2:14, k=1:n_vehicles], sum(x[j,i,k] for j=1:14) == sum(x[i,j,k] for j=2:15))

    # solve model
    optimize!(model)

    # return decision variables
    return value.(x), value.(u)

end

waste_management3 (generic function with 1 method)

In [ ]:
# # specify parameters
# n_vehicles = ceil(n_vehicles_needed/n_individual_per_aggregate) + 2
# n_vehicles = Int64(n_vehicles)
n_individual_per_aggregate = 4

x_opt, u_opt = waste_management3(cost, waste, n_vehicles, n_individual_per_aggregate);

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter TimeLimit to value 300
Set parameter TimeLimit to value 300
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2309 rows, 1920 columns and 10920 nonzeros
Model fingerprint: 0x9a5741ca
Variable types: 120 continuous, 1800 integer (1800 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+04]
  Objective range  [3e+00, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+04]
Presolve removed 1608 rows and 1064 columns
Presolve time: 0.04s
Presolved: 701 rows, 856 columns, 7390 nonzeros
Variable types: 88 continuous, 768 integer (768 binary)

Root relaxation: objective 4.014237e+03, 304 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    Best

In [ ]:
# find where x_opt[:,:,k] is 1
x_opt_1 = findall(x_opt[:,:,7] .== 1)


2-element Vector{CartesianIndex{2}}:
 CartesianIndex(1, 12)
 CartesianIndex(12, 15)

In [ ]:
locations_df

Row,Neighborhood,Area,2020 Households,Centerpoint X,Centerpoint Y
,String31,Int64,Int64,Float64,Float64
1,Landfill,0,0,-71.0864,41.9235
2,East Cambridge,1,6953,-71.079,42.3683
3,MIT,2,806,-71.0914,42.358
4,Wellington-Harrington,3,2931,-71.093,42.371
5,The Port,4,2890,-71.0968,42.3658
6,Cambridgeport,5,5279,-71.1078,42.3592
7,Mid-Cambridge,6,6460,-71.1084,42.3725
8,Riverside,7,4047,-71.1141,42.3674
9,Baldwin,8,1799,-71.1154,42.3807
